In [8]:
from datetime import datetime, date, timedelta
import calendar
import os
import logging
import scrapy
from scrapy.crawler import CrawlerProcess

from bs4 import BeautifulSoup

import requests
import pandas as pd
import numpy as np
from statistics import mode
import plotly.express as px
import plotly.graph_objects as go
import time

import requests
import pandas as pd
from bs4 import BeautifulSoup
import os
import plotly.express as px
import plotly.io as pio

import boto3
from sqlalchemy import create_engine, text

In [9]:
cities = ['Mont Saint Michel', 'St Malo', "Bayeux", "Le Havre", "Rouen", "Paris", "Amiens", "Lille", "Strasbourg", "Chateau du Haut Koenigsbourg", "Colmar", "Eguisheim", "Besancon", "Dijon", "Annecy", "Grenoble", "Lyon", "Gorges du Verdon", "Bormes les Mimosas", "Cassis", "Marseille", "Aix en Provence", "Avignon", "Uzes", "Nimes", "Aigues Mortes", "Saintes Maries de la mer", "Collioure", "Carcassonne", "Ariege", "Toulouse", "Montauban", "Biarritz", "Bayonne", "La Rochelle"]

In [10]:
url = "https://nominatim.openstreetmap.org/search?"
user_agent = "project_kayak/1.0 (mouton.cl@gmail.com)"

def find_gps(address, format='json'):
    params = {"q": address, "format": format}
    headers = {"User-Agent": user_agent}
    response = requests.get(url, params=params, headers=headers)
    
    if response.status_code != 200:
        print(f"Erreur: Réponse avec le code de statut {response.status_code} pour l'adresse '{address}'")
        return None
    
    try:
        return response.json()
    except ValueError:
        print(f"Erreur: Impossible de parser la réponse JSON pour l'adresse '{address}'")
        print(f"Contenu de la réponse: {response.text}")
        return None

columns = ['id', 'name', 'latitude', 'longitude']
data = []

for count, city in enumerate(cities):
    response = find_gps(address = city)
    row =[count, city, response[0]['lat'], response[0]['lon']]
    data.append(row)
    time.sleep(1)
    
df_cities = pd.DataFrame(data=data, columns=columns)

In [11]:
display(df_cities.head())
df_cities.to_csv('files/cities_infos.csv', index=False)

,id,name,latitude,longitude
0,0,Mont Saint Michel,48.6359541,-1.511459954959514
1,1,St Malo,49.314695,-96.9538228
2,2,Bayeux,49.2764624,-0.7024738
3,3,Le Havre,49.4938975,0.1079732
4,4,Rouen,49.4404591,1.0939658


# Weather

In [12]:
api_key = "01c1f72fc7a91c71b390a1cd59948cd0"

In [13]:
# Extraire les coordonnées de la première ville
lat = df_cities.iloc[0]['latitude']
lon = df_cities.iloc[0]['longitude']

# Requête API pour la première ville
response = requests.get(f"https://api.openweathermap.org/data/3.0/onecall?lat={lat}&lon={lon}&units=metric&appid={api_key}")

# Vérifier si la requête a réussi
if response.status_code == 200:
    data = response.json()  # Convertir la réponse en format JSON
    # Maintenant, vous pouvez traiter les données comme vous le souhaitez
    print(data)
else:
    print("La requête a échoué, code d'erreur :", response.status_code)


{'lat': 48.636, 'lon': -1.5115, 'timezone': 'Europe/Paris', 'timezone_offset': 7200, 'current': {'dt': 1721298977, 'sunrise': 1721276580, 'sunset': 1721332871, 'temp': 26.27, 'feels_like': 26.27, 'pressure': 1020, 'humidity': 63, 'dew_point': 18.67, 'uvi': 5.67, 'clouds': 93, 'visibility': 10000, 'wind_speed': 0.89, 'wind_deg': 190, 'wind_gust': 2.24, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}]}, 'minutely': [{'dt': 1721299020, 'precipitation': 0}, {'dt': 1721299080, 'precipitation': 0}, {'dt': 1721299140, 'precipitation': 0}, {'dt': 1721299200, 'precipitation': 0}, {'dt': 1721299260, 'precipitation': 0}, {'dt': 1721299320, 'precipitation': 0}, {'dt': 1721299380, 'precipitation': 0}, {'dt': 1721299440, 'precipitation': 0}, {'dt': 1721299500, 'precipitation': 0}, {'dt': 1721299560, 'precipitation': 0}, {'dt': 1721299620, 'precipitation': 0}, {'dt': 1721299680, 'precipitation': 0}, {'dt': 1721299740, 'precipitation': 0}, {'dt': 1721299800, 

In [14]:
def get_weather(latitude, longitude):
    # We set units = metric to have Celsius temperature instead of Kelvin
    # We only keep information about daily forecast for 8 days 
    url = f"https://api.openweathermap.org/data/3.0/onecall?lat={latitude}&lon={longitude}&units=metric&exclude=current,minutely,hourly,alerts&appid={api_key}"
    response = requests.get(url)
    data = response.json()

    if data :
        humidity = data["daily"][0]["humidity"]                     # % of humidity
        temp_avg = data["daily"][0]["temp"]["day"]                  # average temperature
        temp_min = data["daily"][0]["temp"]["min"]                  # minimum temperature
        temp_max = data["daily"][0]["temp"]["max"]                  # maximum temperature
        temp_perceived = data["daily"][0]["feels_like"]["day"]      # perceived temperature
        wind_speed = data["daily"][0]["wind_speed"]                 # wind speed in meter/seconde
        clouds = data["daily"][0]["clouds"]                         # % of cloudiness
        pop = data["daily"][0]["pop"]                               # probability of precipitation, between 0 and 1
        
        return humidity, temp_avg, temp_min, temp_max, temp_perceived, wind_speed, clouds, pop
    else:
        return None

In [15]:
weather_results = []

for index, row in df_cities.iterrows():
    name = row["name"]
    latitude = row["latitude"]
    longitude = row["longitude"]

    humidity, temp_avg, temp_min, temp_max, temp_perceived, wind_speed, clouds, pop = get_weather(latitude, longitude)

    weather_results.append({"name": name, 
                            "latitude": latitude, 
                            "longitude": longitude, 
                            "Humidity": humidity, 
                            "Temperature_avg": temp_avg,
                            "Temperature_min": temp_min,
                            "Temperature_max": temp_max,
                            "Temperature_perceived": temp_perceived,
                            "Wind_speed": wind_speed,
                            "Cloudiness": clouds,
                            "Precipitation_prob" : round(pop*100) # to have the probability of precipitation in %
                            })

df = pd.DataFrame(weather_results)
df.head()

,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob
0,Mont Saint Michel,48.6359541,-1.511459954959514,62,26.07,15.99,26.87,26.07,3.16,93,0
1,St Malo,49.314695,-96.9538228,61,22.33,12.14,24.61,22.21,4.00,5,0
2,Bayeux,49.2764624,-0.7024738,55,25.31,14.88,26.68,25.33,3.89,96,0
3,Le Havre,49.4938975,0.1079732,51,25.83,16.83,26.51,25.80,3.66,18,20
4,Rouen,49.4404591,1.0939658,52,26.28,15.21,26.43,26.28,3.18,19,0


In [16]:
# Add index column to the dataframe
df = df.reset_index()
df.head()

,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,62,26.07,15.99,26.87,26.07,3.16,93,0
1,1,St Malo,49.314695,-96.9538228,61,22.33,12.14,24.61,22.21,4.00,5,0
2,2,Bayeux,49.2764624,-0.7024738,55,25.31,14.88,26.68,25.33,3.89,96,0
3,3,Le Havre,49.4938975,0.1079732,51,25.83,16.83,26.51,25.80,3.66,18,20
4,4,Rouen,49.4404591,1.0939658,52,26.28,15.21,26.43,26.28,3.18,19,0


In [17]:
# Save the dataframe in a csv file
df.to_csv("files/cities_forecast_weather.csv", index=False)

In [18]:
# Determine what are the best 5 cities destination
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, 
                    cols=3, 
                    start_cell="bottom-left",
                    specs=[[{"type": "domain"}, {"type": "domain"},{"type": "domain"}], 
                    [{"type": "domain"},{"type": "domain"},{"type": "domain"}]],
                    subplot_titles=("Top 5 cities with the least humidity (%)", 
                                    "Top 5 cities with the best day temperature(°C)", 
                                    "Top 5 cities with the least speed wind (m/s)",
                                    "Top 5 cities with the least cloudiness (%)",
                                    "Top 5 cities with the least probability of rain (%)"))

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Humidity']).head(5)["name"],
    values=df.sort_values(by=['Humidity']).head(5)["Humidity"],
    textinfo="label+value"),
    row=1, col=1
)

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Temperature_avg'], ascending=False).head(5)["name"],
    values=df.sort_values(by=['Temperature_avg'], ascending=False).head(5)["Temperature_avg"],
    textinfo="label+value"),
    row=1, col=2
)

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Wind_speed']).head(5)["name"],
    values=df.sort_values(by=['Wind_speed']).head(5)["Wind_speed"],
    textinfo="label+value"),
    row=1, col=3
)

# Ajouter une petite valeur à toutes les valeurs de la catégorie "Cloudiness"
df['Cloudiness_adjusted'] = df['Cloudiness'] + 0.1

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Cloudiness_adjusted']).head(5)["name"],
    values=df.sort_values(by=['Cloudiness_adjusted']).head(5)["Cloudiness_adjusted"],
    textinfo="label+value"),
    row=2, col=1
)

# Ajouter une petite valeur à toutes les valeurs de la catégorie "Precipitation_prob"
df['Precipitation_prob_adjusted'] = df['Precipitation_prob'] + 0.1

fig.add_trace(go.Pie(
    labels=df.sort_values(by=['Precipitation_prob_adjusted']).head(5)["name"],
    values=df.sort_values(by=['Precipitation_prob_adjusted']).head(5)["Precipitation_prob_adjusted"],
    textinfo="label+value"),
    row=2, col=2
)

fig.update_layout(
    # title='Top 5 cities with the least humidity',
    height=700, 
    width=1400,
    showlegend=False
)

fig.show()

In [19]:
# Tri par ordre croissant de la catégorie "Cloudiness"
df_cities_sorted = df.sort_values(by='Temperature_avg', ascending=True)

# Affichage des premières lignes du DataFrame trié
df_cities_sorted.head(10)


,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob,Cloudiness_adjusted,Precipitation_prob_adjusted
1,1,St Malo,49.314695,-96.9538228,61,22.33,12.14,24.61,22.21,4.00,5,0,5.1,0.1
9,9,Chateau du Haut Koenigsbourg,48.249410749999996,7.344320233724503,50,22.96,12.63,24.60,22.62,2.28,0,0,0.1,0.1
32,32,Biarritz,43.471143749999996,-1.552726590666314,74,24.67,19.59,24.86,25.12,4.55,1,0,1.1,0.1
2,2,Bayeux,49.2764624,-0.7024738,55,25.31,14.88,26.68,25.33,3.89,96,0,96.1,0.1
33,33,Bayonne,43.4945144,-1.4736657,73,25.33,18.89,25.53,25.82,4.26,0,0,0.1,0.1
11,11,Eguisheim,48.0447968,7.3079618,50,25.76,15.20,26.67,25.70,2.59,0,0,0.1,0.1
13,13,Dijon,47.3215806,5.0414701,50,25.80,15.14,26.30,25.74,3.09,0,0,0.1,0.1
3,3,Le Havre,49.4938975,0.1079732,51,25.83,16.83,26.51,25.80,3.66,18,20,18.1,20.1
10,10,Colmar,48.0777517,7.3579641,50,25.93,15.32,26.97,25.88,2.65,0,0,0.1,0.1
0,0,Mont Saint Michel,48.6359541,-1.511459954959514,62,26.07,15.99,26.87,26.07,3.16,93,0,93.1,0.1


In [20]:
top5_city=df_cities_sorted[0:5]

In [21]:
top5_city

,index,name,latitude,longitude,Humidity,Temperature_avg,Temperature_min,Temperature_max,Temperature_perceived,Wind_speed,Cloudiness,Precipitation_prob,Cloudiness_adjusted,Precipitation_prob_adjusted
1,1,St Malo,49.314695,-96.9538228,61,22.33,12.14,24.61,22.21,4.00,5,0,5.1,0.1
9,9,Chateau du Haut Koenigsbourg,48.249410749999996,7.344320233724503,50,22.96,12.63,24.60,22.62,2.28,0,0,0.1,0.1
32,32,Biarritz,43.471143749999996,-1.552726590666314,74,24.67,19.59,24.86,25.12,4.55,1,0,1.1,0.1
2,2,Bayeux,49.2764624,-0.7024738,55,25.31,14.88,26.68,25.33,3.89,96,0,96.1,0.1
33,33,Bayonne,43.4945144,-1.4736657,73,25.33,18.89,25.53,25.82,4.26,0,0,0.1,0.1


In [22]:
# Save the dataframe in a csv file
top5_city.to_csv("files/top_5city.csv")

In [28]:
df_final = df_cities_sorted.astype({"latitude": float, "longitude": float})

fig = px.scatter_mapbox(df_final, lat="latitude", lon="longitude",
                  
                            mapbox_style="carto-positron",zoom=4, width = 800,  height =860,title = 'Location of top hotels in 5 different cities',color='Temperature_avg'
                          
            )
fig.show()

## Scrapping

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Lire les données météo
weather_data = pd.read_csv('files/cities_forecast_weather.csv')

# Vérifier les données de weather_data
print("Données météo lues depuis le fichier :")
print(weather_data.head())

# Initialiser l'agent utilisateur pour les requêtes
agent = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36'}

# Construire les URL de recherche pour chaque ville du dataset météo
search_urls = ['https://www.booking.com/searchresults.fr.html?ss='+city+'&nflt=review_score%3D60' for city in weather_data['name']]

# Récupérer les URL des hôtels (25 premiers sur la page de recherche de Booking) pour chaque URL de recherche
hotels_data = []

for url in search_urls:
    page = requests.get(url, headers=agent)
    soup = BeautifulSoup(page.content, "html.parser")
    
    hotel_cards = soup.find_all('div', class_='sr_item')
    
    for card in hotel_cards:
        hotel_data = {}
        
        # Nom de l'hôtel
        hotel_data['name'] = card.find('span', class_='sr-hotel__name').text.strip()
        
        # Adresse de l'hôtel
        address_element = card.find('div', class_='address')
        hotel_data['address'] = address_element.text.strip() if address_element else 'N/A'
        
        # Description de l'hôtel
        description_element = card.find('div', class_='hotel_desc')
        hotel_data['description'] = description_element.text.strip() if description_element else 'N/A'
        
        # Score de l'hôtel
        score_element = card.find('div', class_='bui-review-score__badge')
        hotel_data['score'] = score_element.text.strip() if score_element else 'N/A'
        
        # Bonus ou équipements importants
        bonus_element = card.find('div', class_='facility-badge')
        hotel_data['bonus'] = bonus_element.text.strip() if bonus_element else 'N/A'
        
        # URL de l'hôtel
        url_element = card.find('a', class_='hotel_name_link url')
        hotel_data['url'] = url_element['href'] if url_element else 'N/A'
        
        hotels_data.append(hotel_data)

# Vérifier les données des hôtels extraites
print("\nExemples de données d'hôtels extraites :")
print(hotels_data[:5])

# Créer un DataFrame à partir des données des hôtels
if hotels_data:
    booking_data = pd.DataFrame(hotels_data)
    
    # Ajouter les colonnes supplémentaires de weather_data à booking_data
    booking_data = pd.concat([weather_data] * (len(booking_data) // len(weather_data)), ignore_index=True)
    booking_data = booking_data.drop(['latitude', 'longitude', 'Humidity', 'Temperature_avg', 'Temperature_min', 'Temperature_max', 'Temperature_perceived', 'Wind_speed', 'Cloudiness', 'Precipitation_prob'], axis=1)
    
    # Nettoyer les données textuelles
    for col in ['description', 'score', 'bonus']:
        booking_data[col] = booking_data[col].str.replace(r'\n', ' ').str.replace(r'\xa0', ' ')
    
    # Extraire la latitude et la longitude à partir des coordonnées et supprimer la colonne 'latlon'
    booking_data[['latitude', 'longitude']] = booking_data['latlon'].str.split(',', expand=True)
    booking_data = booking_data.drop('latlon', axis=1)
    
    # Convertir les types de données
    booking_data[['latitude', 'longitude']] = booking_data[['latitude', 'longitude']].astype(float)
    
    # Utiliser une expression régulière pour extraire le score
    booking_data['score'] = booking_data['score'].str.extract(r'(\d+[\.,]?\d*)', expand=False).str.replace(',', '.').astype(float)
    
    print("\nExemple de données finales après traitement :")
    print(booking_data.head())
else:
    print("Erreur : Aucune donnée d'hôtel n'a été extraite.")



In [30]:
# On sauvegarde les données sur les differents hotels

booking_data.to_csv('files/booking_data.csv', index=False)

In [ ]:
booking_data.sample(5)

# STOCKAGE AWS S3

In [46]:
# On cree un bucket s3 Amazon

ACCESS_KEY_ID = os.environ.get('aws_access_key_id')
SECRET_ACCESS_KEY = os.environ.get('aws_secret_access_key')

bucketname = 'booking-mouton-files'

In [ ]:
session = boto3.Session(
    aws_access_key_id= ACCESS_KEY_ID,
    aws_secret_access_key= SECRET_ACCESS_KEY
)

s3 = session.client("s3")

bucket = s3.create_bucket(Bucket=bucketname, ACL='public-read-write', CreateBucketConfiguration={'LocationConstraint': 'eu-west-3'})

In [ ]:

# Chargement des fichiers

s3.Bucket(bucketname).upload_file('files/booking_data.csv','booking_data.csv')
s3.Bucket(bucketname).upload_file('files/cities_forecast_weather.csv','cities_forecast_weather.csv')

In [ ]:
# Telechargement d'un dataset pour tester le upload

booking_data = pd.read_csv(f"https://{bucketname}.s3.eu-west-3.amazonaws.com/booking_data.csv")
booking_data.head()

In [ ]:
login = open("login.txt", "r").readlines()

In [ ]:
# Configuration de RDS

filename = "booking_data.csv"

s3.download_file(bucketname, filename, filename)

data = pd.read_csv(filename)

# Connexion à la base de données MySQL

# engine = create_engine(f"mysql+pymysql://{DBUSER}:{DBPASS}@{DBHOST}:{PORT}/{DBNAME}")
engine = create_engine(f"mysql+pymysql://{login[0].strip()}:{login[1].strip()}@{login[2].strip()}:{login[3].strip()}/{login[4].strip()}")

# Stocker les données dans la base de données
data.to_sql('booking_data', con=engine, index=False, if_exists='replace')

os.remove(filename)

In [ ]:
engine

In [ ]:
weather_data = pd.read_csv('files/cities_weather_data.csv')

In [ ]:
# stocker les données météo et de réservation sous forme de tables SQL

weather_data.to_sql('weather', con=engine, if_exists='replace', index=False)
booking_data.to_sql('booking', con=engine, if_exists='replace', index=False)

In [ ]:
# Obtenir des données météorologiques à partir d'une table SQL

weather_query = text("SELECT * FROM weather WHERE quality > 10 ORDER BY quality DESC LIMIT 5")
weather_selection = pd.read_sql(weather_query, engine)
weather_selection.head()

In [ ]:
# choisir les hôtels dont nous avons besoin en fonction de la météo

hotel_query = text("SELECT * FROM booking WHERE cities in ('Lyon','Paris','Annecy','La Rochelle','Avignon')")
hotel_selection = pd.read_sql(hotel_query, engine)
hotel_selection.head()